# Inistallations and Imports

### For Google Colab .... don't work :(

In [ ]:
# --- for Google Colab --- 
# loading  Kaggle API key
#from google.colab import files
#files.upload()

In [ ]:
# install kjaggle and mount kaggle.json
#!pip install -q kaggle
#! mkdir -p ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

In [ ]:
# load data-file from kaggle competiotion - DON'T WORK 
#! kaggle competitions download -c sf-dst-car-price-prediction-part2

### Load DATA in Google Colab (manually upload from local)

In [ ]:
# Unzip datafile to directory 'input'
# datafile preloaded manually from local disk
#! mkdir input
#! unzip sf-dst-car-price-prediction-part2.zip -d input

In [ ]:
# Install for Colab
#! pip install catboost
#! pip install category_encoders
#! pip install albumentations


### Usual imports, Sklearn, Catboost, Tensorflow, Matplotlib, Seaborn

In [ ]:
import numpy as np 
import pandas as pd
import os, re, PIL, cv2

In [ ]:
#for kaggle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

import warnings
from sklearn import metrics

In [ ]:
from catboost import CatBoostRegressor
import category_encoders as ce
from tqdm.notebook import tqdm

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10         # make default plot size
%config InlineBackend.figure_format = 'svg' # make default plot format svg
%matplotlib inline
import seaborn as sns

In [ ]:
# ---- Path for Kaggle ---
PATH = '../input/sf-dst-car-price-prediction-part2/'
# ---- Path for local ----
# PATH=''
# ----- path for Colab ----
#PATH = '/content/input/'

data = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
sample_submission = pd.read_csv(PATH + 'sample_submission.csv')
pd.set_option('display.max_columns', None)

In [ ]:
SEED=42
np.random.seed(SEED)
TARGET = 'price'

# Function

EDA, FE, etc.

In [ ]:
def find_number(field: str):
    """
    function to take only first numeric data from string
    :param field: string to find number
    :return: number (in string format)
    """

    # mask for search
    p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'

    # main cycle for searching
    if re.search(p, field) is not None:
        for catch in re.finditer(p, field):
            return catch[0]

In [ ]:
def name_separate(item):
    """
    function to separete text value in vector 'name' and return only first part
    :param item: text value from field of vector
    :return: first part of text value like '180 BlueEFFICIENCY 1.8 AT'
    """
    result = item.replace('(','-').replace(')','-').split('-')
    if len(result)==3:
        third = result[2]
    else:
        third = ''

    return result[0]#, result[1], third

In [ ]:
def mape(y_true, y_pred):
    """
    function to calculate Mean Absolute Percentage Error
    """
    return np.mean(np.abs((y_pred-y_true)/y_true))*100

In [ ]:
results = pd.DataFrame()
def feature_imp(dataset, regressor, encoder, test_drop = True, target='price'):
    """
    function for checking feature importance on ML
    :param dataset: dataset for ML
    :param regressor: algorithm of ML
    :param encoder: encoder for categorical vectors
    :param test_drop: boolean flag whether need to separate test data from dataset
    :param target: name of target
    :return: list of features with weights descending, score of algorithm and MAPE score
    """
    global results

    # copy dataset useful if you need to drop some vectors from dataset
    dataset=dataset.copy()

    # checkin boolean flag to drop test data from dataset
    if test_drop:
        dataset = dataset[dataset.test == 0]

    # drop vector 'test'
    # dataset.drop(['test'], axis=1, inplace=True)

    # make variables x and y for ML model
    x, y = dataset.drop(target, axis=1), np.log(dataset[target])

    # encoding categorical vectors with encoder
    encoder.fit(x, y)
    X = encoder.transform(x)

    # split for validate and fit ML model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED)
    regressor.fit(X_train, y_train)

    # make dataframe with features importance
    importance = regressor.feature_importances_
    value = pd.DataFrame(data=importance, index=X.columns, columns=['importance'])
    print(value.importance.sort_values(ascending=False))

    # calculate ML model score
    score = regressor.score(X_test, y_test)
    print(f'Score of regressor {score*100:0.2f}%')

    # calculate MAPE score
    predict = np.exp(regressor.predict(X_test))
    print(f"MAPE: {(mape(np.exp(y_test), predict)):0.2f}%")

    if not results.empty:
        results.drop(columns=results.columns, inplace=True)
    results = X_test.copy()
    results['y_test'] = np.exp(y_test)
    results['predict'] = predict.astype('int64')

In [ ]:
def cooking():
    global data, test

    # ---- load datasets ---
    data = pd.read_csv(PATH + 'train.csv')
    test = pd.read_csv(PATH + 'test.csv')
    sample_submission = pd.read_csv(PATH + 'sample_submission.csv')

    # --- bodyType ----
    ## -- repair mistake in data
    data.loc[(data.bodyType == 'хэтчбек 3 дв.') & (data.numberOfDoors == 2), 'numberOfDoors'] = 3

    ## -- working
    n_doors = {
        'внедорожник 3 дв.': 'внедорожник',
        'внедорожник 5 дв.': 'внедорожник',
        'внедорожник открытый': 'внедорожник открытый',
        'кабриолет': 'кабриолет',
        'компактвэн': 'компактвэн',
        'купе': 'купе',
        'купе-хардтоп': "купе-хардтоп",
        'лимузин': 'лимузин',
        'лифтбек': 'лифтбек',
        'минивэн': 'минивэн',
        'пикап двойная кабина': 'пикап двойная кабина',
        'родстер': 'родстер',
        'седан': 'седан',
        'седан 2 дв.': 'седан',
        'универсал 5 дв.': 'универсал',
        'хэтчбек 3 дв.': 'хэтчбек',
        'хэтчбек 5 дв.': 'хэтчбек'
    } # dic of clear body types
    data['body'] = data.bodyType.map(n_doors)
    test['body'] = test.bodyType.map(n_doors)
    data['bodyDoors'] = data.apply(lambda q: str(q.body) + " " + str(q.numberOfDoors), axis=1)
    test['bodyDoors'] = test.apply(lambda q: str(q.body) + " " + str(q.numberOfDoors), axis=1)

    # --- engineDisplacement ---
    data['engineDisplacementValue'] = data.engineDisplacement.apply(find_number)
    test['engineDisplacementValue'] = test.engineDisplacement.apply(find_number)
    data.engineDisplacementValue.fillna(value=0.0001, inplace=True)
    test.engineDisplacementValue.fillna(value=0.0001, inplace=True)
    data.engineDisplacementValue = data.engineDisplacementValue.apply(float)
    test.engineDisplacementValue = test.engineDisplacementValue.apply(float)
    scaler = MinMaxScaler()
    data['edvMinMax'] = scaler.fit_transform(data[['engineDisplacementValue']])
    test['edvMinMax'] = scaler.transform(test[['engineDisplacementValue']])
    
    data['displacement']=data.engineDisplacementValue//1
    test['displacement']=test.engineDisplacementValue//1
    
    # --- enginePower ---
    data['enginePowerValue'] = data.enginePower.apply(find_number).apply(int)
    test['enginePowerValue'] = test.enginePower.apply(find_number).apply(int)

    data['epvLog'] = data.enginePowerValue.apply(np.log)
    test['epvLog'] = test.enginePowerValue.apply(np.log)

    
    # --- productionDate ---
    # till 3 years - free technical support 
    # till 5 years - guarantee period 
    # after 10 years - vehicle loose up to half cost 
    # after 30 years - it is just rare 
    age = lambda date: 3 if date >= 2018 \
        else 5 if date >=2016 else 10 if date >= 2011 \
        else 20 if date>=2001 else 30

    data['age']=data.productionDate.apply(age)
    test['age']=test.productionDate.apply(age)    
    
    
    # --- modelDate ---
    data['modelDateLog']=data.modelDate.apply(lambda q: np.log(2022-q))
    test['modelDateLog']=test.modelDate.apply(lambda q: np.log(2022-q))
    scaler = MinMaxScaler()
    data['modelDateMM'] = scaler.fit_transform(data[['modelDate']])
    test['modelDateMM'] = scaler.transform(test[['modelDate']])

    # --- owners ---
    data['Владельцы'].fillna('2 владельца', inplace=True)
    data['owners'] = data['Владельцы'].apply(find_number)
    data['ownerValue'] = data.apply(lambda q: int(q.owners), axis=1)
    data['Владение'].fillna('Не известно', inplace=True)

    test['Владельцы'].fillna('2 владельца', inplace=True)
    test['owners']=test['Владельцы'].apply(find_number)
    test['ownerValue'] = test.apply(lambda q: int(q.owners), axis=1)
    test['Владение'].fillna('Не известно', inplace=True)

    # --- delete noise data ---
    ejection={
        'color': {'розовый'},
        'engineDisplacement': {'3.4 LTR', '4.6 LTR', '4.9 LTR', '5.6 LTR', '6.3 LTR'},
        'sell_id': {1100083262},
        'Руль': {'Правый'},
        'bodyType': {'седан 2 дв.', 'компактвэн', 'лимузин'}
    }
    for vector, values in ejection.items():
        for value in values:
            data = data[data[vector] != value]

    # ---name---
    data['name1'] = data.name.apply(name_separate)
    data['vehicle'] = data.brand + ' ' + data.model_info + ' ' + data.vehicleConfiguration + ' ' + data.name1

    test['name1'] = test.name.apply(name_separate)
    test['vehicle'] = test.brand + ' ' + test.model_info + ' ' + test.vehicleConfiguration + ' ' + test.name1

    # --- description ---
    data['descriptionLen'] = data.description.apply(lambda q: len(q.split()))
    test['descriptionLen'] = test.description.apply(lambda q: len(q.split()))

cooking()

In [ ]:
list_to_drop = ['engineDisplacementValue','modelDate','modelDateMM',
                'Руль',
                'enginePowerValue',
                'description',
                ]
parameters = {
    'regressor': CatBoostRegressor(random_state=SEED),
    'encoder': ce.TargetEncoder()
}
feature_imp(data.drop([], axis=1), test_drop=False, **parameters)

# Catboost submission

In [ ]:
x, y = data.drop([TARGET], axis=1), np.log(data[TARGET])
q = test
encoder=ce.TargetEncoder()
encoder.fit(x, y)
x = encoder.transform(x)
q = encoder.transform(q)

In [ ]:
model = CatBoostRegressor(iterations = 5000,random_seed = SEED,
                          eval_metric='MAPE',custom_metric=['R2', 'MAE'])
model.fit(x, np.log(y), verbose_eval=0)

model.save_model('catboost_single_model_EDA1.model')

q['price'] = np.exp(model.predict(q))

In [ ]:
catboost_prediction = q[['sell_id', 'price']]
catboost_prediction.to_csv('catboost_sub.csv', index=False)
catboost_prediction
# 11.6153% on leaderboard

# Computer vision

### Image preprocessinп

In [ ]:
plt.figure(figsize = (12,8))

random_image = data.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
# split data for train and test parts
x, y, q = data.drop(TARGET, axis=1), data[TARGET], test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True, random_state=SEED)
x_sub = test

text_train = x_train.description
text_test = x_test.description
text_sub = test.description

x, q = x.drop('description', axis=1), q.drop('description', axis=1)
x_train = x_train.drop('description', axis=1)
x_test = x_test.drop('description', axis=1)

encoder=ce.TargetEncoder().fit(x,y)
x, q = encoder.transform(x), encoder.transform(q)
x_train, x_test = encoder.transform(x_train), encoder.transform(x_test)

In [ ]:
size = (320, 240)

data_all = data.append(test)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data_all['sell_id'].iloc[index].values):
        image = cv2.imread(PATH + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(x_train.index)
images_test = get_image_array(x_test.index)
images_sub = get_image_array(x_sub.index)

### Augmentation function

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    GaussNoise, MotionBlur, MedianBlur, 
    #IAAPiecewiseAffine, IAASharpen, IAAEmboss,   # this aurmentations don't work in kaggle, but work in Colab
    PiecewiseAffine, Sharpen, Emboss,           # This aurmentations work in kaggle, but do not work in Colab
    RandomBrightnessContrast, Flip, OneOf, Compose
)

augmentation = Compose([
    HorizontalFlip(),
    GaussNoise(p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),   # 0.1
    #    IAAPiecewiseAffine(p=0.3), # 0.3
        PiecewiseAffine(p=0.3), # 0.3
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
    #    IAASharpen(),
    #    IAAEmboss(),
        Sharpen(),
        Emboss(),
        RandomBrightnessContrast(),
    ], p=0.3),
    HueSaturationValue(p=0.3),
], p=1)

In [ ]:
def make_augmentations(images):
  print('применение аугментаций', end = '')
  augmented_images = np.empty(images.shape)
  for i in range(images.shape[0]):
    if i % 200 == 0:
      print('.', end = '')
    augment_dict = augmentation(image = images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
  print('')
  return augmented_images

### Example preprocessing images

In [ ]:
#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[31])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

### Preprocessing Data
* tokenizing text,  
* split to test and train
* function to make tuple from images

In [ ]:
# TOKENIZER
MAX_WORDS = 100000          # The maximum number of words to be used. (most frequent)
MAX_SEQUENCE_LENGTH = 256   # Max number of words in each complaint.

In [ ]:
# NLP part
#data_all=data.append(test)
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data_all.description)
#data_all = x.append(q)


In [ ]:
train_index, test_index = x_train.index, x_test.index

In [ ]:
encoder=ce.TargetEncoder().fit(x,y)
x, q = encoder.transform(x), encoder.transform(q)
x_train, x_test = encoder.transform(x_train), encoder.transform(x_test)

scaler = StandardScaler().fit(x)
x_train, x_test, q = scaler.transform(x_train), scaler.transform(x_test), scaler.transform(q)
x = scaler.transform(x)
x_sub=q

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, x_train, text_train, y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, x_test, text_test, y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(x_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, x_sub, text_sub, y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

### Neuro Net Models

In [ ]:
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [ ]:
# Fine tuning
#-------------
efficientnet_model.trainable = True

fine_tune_at = len(efficientnet_model.layers)//2

for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False                                # freeze learning in this leyers

print(len(efficientnet_model.layers))
for layer in efficientnet_model.layers:                     # check freezed and free to learn layers
    print(layer, layer.trainable)

In [ ]:
# tab1 , without top
tab_start = tf.keras.Input(shape=x_train.shape[1])

tab_flow1 = Sequential()
tab_flow1.add(L.Dense(256, activation="sigmoid"))
tab_flow1.add(L.BatchNormalization(axis=1))
tab_flow1.add(L.Dropout(0.5))
tab_flow1.add(L.Dense(256, activation="relu"))
tab_flow1.add(L.BatchNormalization(axis=1))
tab_flow1.add(L.Dropout(0.5))
tab_flow1.add(L.Dense(256, activation="relu"))
tab_flow1.add(L.BatchNormalization(axis=1))
tab_flow1.add(L.Dropout(0.25))
#tab_flow1.add(L.Dense(1, activation="sigmoid"))

tab_flow2 = L.Dense(1, activation="linear")

tab_flow3 = L.Dense(1, activation="sigmoid")
tab_flow4 = L.Dense(1, activation="sigmoid")

tab_result = tab_flow1(tab_start) + tab_flow2(tab_start) + tab_flow3(tab_start) + tab_flow4(tab_start)

model_tab = Model(inputs=tab_start, outputs=tab_result, name='Tabular')

In [ ]:
model_nlp = Sequential(name='NLP')
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(128, activation="sigmoid"))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [ ]:
#объединяем выходы трех нейросетей
multyInput = L.concatenate([efficientnet_output, model_tab.output, model_nlp.output])

# being our regression head
funnel = L.Dense(256, activation="relu")(multyInput)
funnel = L.Dense(1,)(funnel)

model_multy = Model(inputs=[efficientnet_model.input, model_tab.input, model_nlp.input], outputs=funnel)
model_multy.summary()

### Multy Input Model Training

In [ ]:
# load weights preloadrd from local disk
# model_multy.load_weights('/content/nn_final.hdf5')
model_multy.load_weights('../input/nn-final-2/nn_final_kaggle2.hdf5')

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.005)
model_multy.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')  #kaggle
#checkpoint = ModelCheckpoint('/content/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')     #colab
earlystop = EarlyStopping(monitor='val_MAPE', patience=20, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [ ]:
history = model_multy.fit(train_dataset.batch(25),
                    epochs=15,
                    validation_data = test_dataset.batch(25),
                    callbacks=callbacks_list
                   )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [ ]:
# Colab
#model_multy.load_weights('/content/best_model.hdf5')
#model_multy.save('/content/nn_final_colab.hdf5')

# Kaggle
model_multy.load_weights('../working/best_model.hdf5')
model_multy.save('../working/nn_final_kaggle.hdf5')

In [ ]:
test_predict_nn3 = model_multy.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0])):0.2f}%")

In [ ]:
sub_predict_nn3 = model_multy.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

# Blend ML and DL

In [ ]:
blend_predict = (catboost_prediction.price + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict)):0.2f}%")

In [ ]:
# best blend is 70 / 30 
blend_sub_predict = 0.7*np.exp(catboost_prediction.price)+0.3*sub_predict_nn3[:,0]
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)
#11.37

In [ ]:
np.exp(catboost_prediction.price)+sub_predict_nn3[:,0]

In [ ]:
sub_predict_nn3[:,0]